# **Cat and Dog Image Project Report**


### **Abstract**

### **Introduction**

The cat-dog image project explores the field of domain specific image classification. This task is preformed quickly and accurately by humans, however remains difficult for computers. The signifigance of this project is to help develop methods of classifying subjects that possess many the ineherent similarites (4 legs, hairy bodies, similar face structure) and exist in similar environments (inside houses, on grass, next to humans). We adopt the tools of manual image feature extraction to classify images in our dataset. Additionally we apply neural networks (CNNs and a pretrained ResNet) to solve the task given their ability to learn which features matter when classifying the images. We preprocess the data, propose several feature extracting techniques (BRISK and HOG) and ML models (KNN, SVM, RF) and compare our results to the NN methods.

### **Background**

### **Data**

Our data gathered here (https://www.microsoft.com/en-us/download/details.aspx?id=54765) consists of 25,000 images balanced evenly between cats and dogs. This Kaggle data is a subset of data gathered for Microsoft Research through Asirra (Animal Species Image Recognition for Restricting Access) that works by asking users to identify photographs of cats and dogs.  

We built our training set out of the first 11,000 images of each catergory (22,000 total).
We built our validation set out of the next 1,000 images of each category (2,000 total).
We built our testing set out of the last 500 images of each category (1,000 total). 

Our preprocessing steps included image resizing to a standard 128x128 pixels and normalizing pixel values.

In the preprocessing step, some empty or small image files we discovered and discarded from the training set, in total 6 each of cats and dogs were removed. 

### **Methods**

BVOW - (Bag of Visual Words) is a method of image classification that builds a "vocabulary" of BRISK features to classify an image. When used for binary image classification, histograms of features are created to represent each image and a comparison between the classes using a KNN classifier is applied. The output is a predictor that will extract the feature "vocabulary" from the inferenced image and return a class prediction.


The HSV (Hue, Saturation, Value) Hue space extracts color representations measured in degrees where 0° is red, 120° is green, and 240° is blue. Saturation represents the intensity or purity of the color and is measured from 0 to 100%, and value represents the brightness of the color. This characteristic is useful for determining image borders.

### **Evaluation**

### **Conclusion**